In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

%cd /content/gdrive/My\ Drive/Accident_Anticipation/depth_estimation/google_distance_code/google-research/depth_from_video_in_the_wild
from image_utils import *

%cd /content/gdrive/My\ Drive/Accident_Anticipation/depth_estimation/google_distance_code/google-research

!pip install tensorflow-graphics

import warnings

warnings.filterwarnings('ignore')

from __future__ import absolute_import, division, print_function
from absl import logging
import os
# import image_utils
import cv2
import re
import time
import glob
from matplotlib import pyplot as plt
import pickle
import numpy as np
from PIL import Image
import pandas as pd
from google.colab.patches import cv2_imshow

import tensorflow as tf
from depth_from_video_in_the_wild import model

# https://github.com/google-research/google-research/tree/master/depth_from_video_in_the_wild

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/Accident_Anticipation/depth_estimation/google_distance_code/google-research/depth_from_video_in_the_wild
/content/gdrive/My Drive/Accident_Anticipation/depth_estimation/google_distance_code/google-research
     |████████████████████████████████| 245kB 2.9MB/s 


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Feature and Image Paths:

In [0]:
cropped_img_path = '/content/gdrive/My Drive/Accident_Anticipation/depth_estimation/google_distance_code/accident_data/reformatting_imgs/reordered_final_outputs_2/'
optimal_crop_path = '/content/gdrive/My Drive/Accident_Anticipation/Anticipating-Accidents/preprocessed_features/optimal_crops_det_2/' 
# path to depth_from_the_wild outputs
path_odom = '/content/gdrive/My Drive/Accident_Anticipation/depth_estimation/google_distance_code/google-research/depth_from_video_in_the_wild/odometry_trajectory_outputs/'
path_depth = '/content/gdrive/My Drive/Accident_Anticipation/depth_estimation/depth_maps/distance_maps/'

root = '/content/gdrive/My Drive/Accident_Anticipation/Anticipating-Accidents'
# path
train_path = root + '/dataset/features/training/' 
test_path = root +  '/dataset/features/testing/' 

###
batch_str = 'batch_045.npz'
mode = 'training'
vid_num = 2
###

num1 = (126, 46)
org_bb = np.load(train_path+batch_str) # use org_bb to retrieve bounding boxes, and the correct video ID
resized_bb = np.load(optimal_crop_path+'training/'+batch_str)['new_det']
# track_ids = np.load(path_track_ids+'training/'+batch_str)
frame_paths, frame_labels = retrieve_dir1(org_bb['labels'], org_bb['ID'],mode, num1[0])
odom_complete_paths = retrieve_dir_custom(path_odom, org_bb, 'training', num1[0], split_type=1)
depth_complete_paths = retrieve_dir_custom(path_depth, org_bb, 'training', num1[0])

depth_np = np.load(depth_complete_paths[vid_num]) # ['distance_map']
odom_np = np.load(odom_complete_paths[vid_num]) # ['rot', 'trans', 'instrinsic_matrix', 'orientation', 'position']

cur_vid_folder = re.split('/', frame_paths[vid_num])[-3]+'_'+re.split('/', frame_paths[vid_num])[-2]
cur_img_path = cropped_img_path+'/'+'training'+'/' + cur_vid_folder
# create list
folders = sorted(os.listdir(cur_img_path))
img_list = [cur_img_path +'/' + f for f in folders if 'png' in f and 'seg' not in f]

In [0]:
new_save_path = '/content/gdrive/My Drive/Accident_Anticipation/EXTRACT_PHYSICS_NOTEBOOKS/physical_features/5_world_coordinates_4v/'

num1 = (126, 46)

def retrieve_global_coord(global_df_path, feat_path, batch_str, num_samp, mode='training'):
  glob_feat = np.zeros((10,100, 20, 4))
  org_bb = np.load(feat_path+batch_str) # use org_bb to retrieve bounding boxes, and the correct video ID
  frame_paths, frame_labels = retrieve_dir1(org_bb['labels'], org_bb['ID'], mode, num_samp)
  global_df_paths = retrieve_dir_custom(new_save_path, org_bb, mode, num_samp, split_type=2)
 
  for batch_num in range(10):
    cur_df_path = global_df_paths[batch_num]
    # add try except functionallity in case df doesn't exisit
    try:
      trajectory_df = pd.read_pickle(cur_df_path)
      time_stamps = np.linspace(0,5,100) # use time_stamps to retrieve the correct index
      frames = trajectory_df.groupby('TIMESTAMPS')

      for group_name, group_data in frames:
        time_ind = np.argwhere(time_stamps == group_name)[0][0] 
        cur_x, cur_y = group_data['X'], group_data['Z']
        v_x, v_y = group_data['v_x'], group_data['v_y']
        glob_feat[batch_num, time_ind,0:len(cur_x),0] = cur_x
        glob_feat[batch_num, time_ind,0:len(cur_x),1] = cur_y
        glob_feat[batch_num, time_ind,0:len(cur_x),2] = v_x
        glob_feat[batch_num, time_ind,0:len(cur_x),3] = v_y
        # 4 dimensions
        # 4 velocity dimensions
    except:
      print("Dataframe doesn't exist: " + str(cur_df_path))


  # world_coordinate modification
  # glob_feat = take_out_nans(glob_feat)
  # glob_feat[glob_feat == np.inf] = 0
  # glob_feat[glob_feat*(-1) == np.inf] = 0
  # glob_feat = glob_feat*0.1  ## change/normalize the global features

  return glob_feat 

def take_out_nans(phys_feat):
    if np.sum(np.isnan(phys_feat)) > 0:
      nan_indices = np.argwhere(np.isnan(phys_feat))
      for k in range(nan_indices.shape[0]):
        phys_feat[nan_indices[k,0],nan_indices[k,1],nan_indices[k,2],nan_indices[k,3]] = 0
    return phys_feat

def show_agents_visualization(resized_bb, cropped_img_path, vid_num, batch_str, frame_paths, trajectory_df, k_width = 12, mode= 'training'):
  cur_vid_folder = re.split('/', frame_paths[vid_num])[-3]+'_'+re.split('/', frame_paths[vid_num])[-2]
  cur_img_path = cropped_img_path+'/'+mode+'/' + cur_vid_folder
  folders = sorted(os.listdir(cur_img_path))
  img_list = [cur_img_path +'/' + f for f in folders if 'png' in f and 'seg' not in f]
  bboxes = resized_bb[vid_num]
  crop_type = np.max(np.max(bboxes[:,:,-2]))
  str_path = cropped_img_path+mode
  folders = sorted(os.listdir(str_path))
  # rgb_path = '../../Anticipating-Accidents/dataset/videos/frames/'+a1 +'/'+a2 +'/'+cur_file + '/'
  # video_path =  '../../Anticipating-Accidents/dataset/videos/'+a1 +'/'+a2 +'/'+cur_file +'.mp4'
  fig, ax = plt.subplots()
  max_agent_ind = np.max(bboxes[:,:,-1])
  frames = trajectory_df.groupby('Agent_ID')

  for img_ind in range(98):
      cur_img_pth = img_list[img_ind]
      # crop image to dimensions
      # cur_img_pth = rgb_path +'frame'+str(img_ind)+'.jpg' ############ change this name
      frame = cv2.imread(cur_img_pth)
      frame = frame[:, 0:416, :] # crop images, since three are fused together for training purposes
      new_bboxes = bboxes[img_ind]
      new_bboxes = new_bboxes[~np.all(new_bboxes == 0, axis=-1)] ## take out zero values from new_bboxes  
      #############################
      ## Replace with plotting function
      agents_ids = new_bboxes[:,-1]

      ax = _add_traj(frames, img_ind, ax, k_width = k_width)
      fig.canvas.draw()
      img = np.fromstring(fig.canvas.tostring_rgb(), dtype=np.uint8,sep='')
      img = img.reshape(fig.canvas.get_width_height()[::-1] + (3,))
      img = cv2.cvtColor(img,cv2.COLOR_RGB2BGR)

      font = cv2.FONT_HERSHEY_SIMPLEX # change font type?
  #     frame = cv2.imread(cur_img_pth)
      for num_box in range(new_bboxes.shape[0]):
          cv2.rectangle(frame,(int(new_bboxes[num_box,0]),int(new_bboxes[num_box,1])),(int(new_bboxes[num_box,2]),int(new_bboxes[num_box,3])),(0,255,0),1)
          # add agent ids in top right-hand corner
          cv2.putText(frame,str(new_bboxes[num_box,-1]),(int(new_bboxes[num_box,0]),int(new_bboxes[num_box,1])), font, 0.3,(0,0,255),1,cv2.LINE_AA) ## put physical parameters here

      top_left_label = str(img_ind) + ': ' + str(crop_type) +', ' + cur_vid_folder
      cv2.putText(frame,top_left_label ,(10,30), font, 0.25,(255,255,255),1)
      # cv2_imshow(frame)
      plt_img = cv2.resize(img,(400,200))
      plt_img = cv2.hconcat([plt_img, plt_img])
      frame_resized = cv2.resize(frame, (800, 200))
      cv2_imshow(cv2.vconcat([plt_img, frame_resized]))
  #         plt.show()


def _add_traj(frames, k, ax, k_width = 12):
    time_stamps = np.linspace(0,5,100)
    min_x, max_x = np.min(np.min(frames['X'])), np.max(np.max(frames['X']))
    min_z, max_z = np.min(np.min(frames['Z'])), np.max(np.max(frames['Z']))
    ax.clear()
    ax.set_xlim(min_x, max_x)
    ax.set_ylim(min_z, max_z)
    ax.set_title('Frame '+ str(k))
    for group_name, group_data in frames:
      past_k = k - k_width
      if past_k < 0:
        past_k = 0
      mask = (group_data['TIMESTAMPS'].values> time_stamps[past_k])*(group_data['TIMESTAMPS'].values < time_stamps[k])
      true_indices = [i for i, x in enumerate(mask) if x]
      if len(true_indices)>0:
        new_mask = mask
        new_mask[true_indices] = False
        new_mask[true_indices[-1]] = True
      else:
        new_mask = mask
      if group_name != 0:
        cor_x = group_data["X"][mask].values
        cor_y = group_data["Z"][mask].values
        ax.plot(
        cor_x,
        cor_y,
        ".")
        # if len(true_indices)>0:
        #   p_x = np.asscalar(group_data["X"][new_mask].values)
        #   p_y = np.asscalar(group_data["Z"][new_mask].values)
        #   v_x = np.asscalar(group_data["v_x"][new_mask].values)
        #   v_y = np.asscalar(group_data["v_y"][new_mask].values)
        #   print('Inputs:')
        #   print(p_x, p_y,v_x, v_y)
        #   ax.arrow(
        #   p_x,
        #   p_y,
        #   v_x,
        #   v_y)
      else:
        ego_x = group_data["X"][0:k].values
        ego_y = group_data["Z"][0:k].values
        ax.plot(
        ego_x,
        ego_y,
        "-") 
    return ax

def full_visualization(det, vid_num, batch_str, frame_paths, mode= 'training'):
  resized_bb = det
  cur_img_path = frame_paths[vid_num]
  print(cur_img_path)
  img_list = sort_dir(os.listdir(cur_img_path))
  print(img_list)
  bboxes = resized_bb[vid_num]
  crop_type = np.max(np.max(bboxes[:,:,-2]))

  for img_ind in range(98):
      cur_img_pth = cur_img_path+img_list[img_ind]
      print(cur_img_path)
      frame = cv2.imread(cur_img_pth)
      new_bboxes = bboxes[img_ind]
      new_bboxes = new_bboxes[~np.all(new_bboxes == 0, axis=1)] ## take out zero values from new_bboxes  
      agents_ids = new_bboxes[:,-1]
      font = cv2.FONT_HERSHEY_SIMPLEX # change font type?

      for num_box in range(new_bboxes.shape[0]):
          # print(img_ind)
          # print(new_bboxes[num_box,0],new_bboxes[num_box,1],new_bboxes[num_box,2],new_bboxes[num_box,3] )
          cv2.rectangle(frame,(int(new_bboxes[num_box,0]),int(new_bboxes[num_box,1])),(int(new_bboxes[num_box,2]),int(new_bboxes[num_box,3])),(0,255,0),2)
          # add agent ids in top right-hand corner
          cv2.putText(frame,str(new_bboxes[num_box,-1]),(int(new_bboxes[num_box,0]),int(new_bboxes[num_box,1])), font, 0.5,(0,0,255),2,cv2.LINE_AA) ## put physical parameters here

      top_left_label = str(img_ind) + ': ' + str(crop_type) +', ' + cur_vid_folder
      cv2.putText(frame,top_left_label ,(10,30), font, 0.5,(255,255,255),2)
      frame_resized = cv2.resize(frame, (800, 400))
      cv2_imshow(frame_resized)

In [0]:
batch_str = 'batch_060.npz'
mode = 'training'
vid_num =5

num1 = (126, 46)
org_bb = np.load(train_path+batch_str) # use org_bb to retrieve bounding boxes, and the correct video ID
frame_paths, frame_labels = retrieve_dir1(org_bb['labels'], org_bb['ID'],mode, num1[0])
pkl_str = re.split('/',frame_paths[vid_num])[-3]+'_'+re.split('/',frame_paths[vid_num])[-2]+'.pkl'
# save_path = '/content/gdrive/My Drive/Accident_Anticipation/EXTRACT_PHYSICS_NOTEBOOKS/physical_features/5_world_coordinates_2/'
save_path = '/content/gdrive/My Drive/Accident_Anticipation/EXTRACT_PHYSICS_NOTEBOOKS/physical_features/5_world_coordinates_3v/'
cropped_img_path = '/content/gdrive/My Drive/Accident_Anticipation/depth_estimation/google_distance_code/accident_data/reformatting_imgs/reordered_final_outputs_2/'
optimal_crop_path = '/content/gdrive/My Drive/Accident_Anticipation/Anticipating-Accidents/preprocessed_features/optimal_crops_det_2/' 

training_path = '/content/gdrive/My Drive/Accident_Anticipation/Anticipating-Accidents/dataset/features/training-sort/output_sort9AGE=2IOU=0.3/'
testing_path = '/content/gdrive/My Drive/Accident_Anticipation/Anticipating-Accidents/dataset/features/testing-sort/output_sort10AGE=2IOU=0.3/'


trn_path = '/content/gdrive/My Drive/Accident_Anticipation/Anticipating-Accidents/dataset/features/training-sort/output_Sort2bAGE=3IOU=0.15NPZSort/'

trn_batch = np.load(training_path+'/'+batch_str)
###! DATA IS VISUALIZED HERE!!!!
mod_bb = np.load(optimal_crop_path+'/'+mode+'/'+batch_str)

# file = open(save_path+pkl_str, 'rb')
# trajectory_df = pickle.load(file)
# file.close()

trajectory_df = pd.read_pickle(save_path+pkl_str)

show_agents_visualization(mod_bb['new_det'], cropped_img_path, vid_num, batch_str, frame_paths, trajectory_df, k_width = 20, mode= 'training')

Output hidden; open in https://colab.research.google.com to view.